In [52]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import dialogflow
from telegram import Update
import os
from tqdm.notebook import tqdm
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
import annoy
from gensim.models import Word2Vec, FastText
import numpy as np
import pickle
import pandas as pd

tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [14]:
# Преобразование файла аопросов-ответов в строчный вид
if not os.path.isfile('prepared_answers.txt'):
    
    question = None
    written = False
    
    with open("prepared_answers.txt", "w", encoding="utf8") as fout:  
        with open("Otvety.txt", "r", encoding="utf8") as fin:
            for line in tqdm(fin):
                if line.startswith("---"):
                    written = False
                    continue
                if not written and question is not None:
                    fout.write(question.replace("\t", " ").strip() + "===" + line.replace("\t", " "))
                    written = True
                    question = None
                    continue
                if not written:
                    question = line.strip()
                    continue

In [15]:
# Препроцессинг текста
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [16]:
# Функция для очистки текста из статьи https://habr.com/ru/articles/738176/ адаптированная под русский язык
def clean_text(input_text):

    # HTML-теги: первый шаг - удалить из входного текста все HTML-теги
    clean_text = re.sub('<[^<]+?>', '', input_text)

    # URL и ссылки: далее - удаляем из текста все URL и ссылки
    clean_text = re.sub(r'http\S+', '', clean_text)

    #Эмоджи и эмотиконы: используем собственную функцию для преобразования эмоджи в текст
    #Важно понимать эмоциональную окраску обрабатываемого текста
    clean_text = emojis_words(clean_text)

    # Приводим все входные данные к нижнему регистру
    clean_text = clean_text.lower()

    # Убираем все пробелы
    # Так как все данные теперь представлены словами - удалим пробелы
    clean_text = re.sub('\s+', ' ', clean_text)

    #Убираем специальные символы: избавляемся от всего, что не является "словами"
    clean_text = re.sub('[^а-яА-ЯёЁ0-9\s]', '', clean_text) #

    # Записываем числа прописью: 100 превращается в "сто" (для компьютера)# не работает
    temp = inflect.engine()
    words = []
    for word in clean_text.split():
        if word.isdigit():
            words.append(num2words(int(word), lang='ru'))
        else:
            words.append(word)
    clean_text = ' '.join(words)

        # Стоп-слова: удаление стоп-слов - это стандартная практика очистки текстов
    stop_words = set(stopwords.words('russian'))
    tokens = word_tokenize(clean_text)
    tokens = [token for token in tokens if token not in stop_words]
    clean_text = ' '.join(tokens)

    # Знаки препинания: далее - удаляем из текста все знаки препинания
    clean_text = re.sub(r'[^\w\s]', '', clean_text)

    # И наконец - возвращаем очищенный текст
    return clean_text

# Функция для преобразования эмоджи в слова
def emojis_words(text):

    # Модуль emoji: преобразование эмоджи в их словесные описания
    clean_text = emoji.demojize(text, delimiters=(" ", " "))

    # Редактирование текста путём замены ":" и" _", а так же - путём добавления пробела между отдельными словами
    clean_text = clean_text.replace(":", "").replace("_", " ")

    return clean_text

def preproc_text(text):
    res = str(text).strip()
    res = re.sub(r"\s+", " ", res)
    return res

def build_data(data_q, data_ans):
    data = []
    for idx, texts in enumerate(data_q):
        question = preproc_text(texts)
        answer = preproc_text(data_ans.iloc[idx])
        res = '\nx:' + question + '\ny:' + answer
        data.append(res)
    return data

In [17]:
import mmap
import re

def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [18]:
# Обработка текста

sentences = []
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
exclude = {}
c = 0

file_path_from = 'prepared_answers.txt'
file_path_to = 'Otvety2.txt'

if not os.path.isfile(file_path_to):
    
    N = get_num_lines(file_path_from)
    with open(file_path_to, mode = 'w', encoding="utf8") as fileto:
        with open(file_path_from, encoding="utf8") as filefrom:
            for k in tqdm(range(N)):
                line = filefrom.readline()
                if line == '': break
                spls = preprocess_txt(line)
                sentences.append(spls)
                c += 1
                if c > 50_000:
                    break
                fileto.write(' '.join(spls)+'\n')
    filefrom.close()
    fileto.close()

  0%|          | 0/1163342 [00:00<?, ?it/s]

In [19]:
# Загрузить результат

sentences = []

file_path_from = 'Otvety2.txt'
if os.path.isfile(file_path_from):  
    N = get_num_lines(file_path_from) 
    with open(file_path_to, mode = 'r', encoding="utf8") as filefrom:
        for k in tqdm(range(N)):
            line = filefrom.readline()
            if line == '': break
            sentences.append(line.split())
    filefrom.close()

  0%|          | 0/50000 [00:00<?, ?it/s]

In [20]:
vec = []
_ = [vec.extend(x)  for x in sentences[:100]]
vec = list(set(vec))
vec.sort()
vec[20:30]

['%',
 '(',
 '(+)',
 '(205—239),',
 '(28—75)',
 '(299—325)',
 '(439—472)',
 '(452—498)',
 '(534—560)',
 '(5строк)']

In [22]:
# Обучим модель FastText

file_path_from = 'ft_model'
if not os.path.isfile(file_path_from):  
    
    sentences = [i for i in tqdm(sentences) if len(i) > 2]
    modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)
    modelFT.save("ft_model")
    ft_index = annoy.AnnoyIndex(100 ,'angular')

In [23]:
# Загрузить модель

modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

In [25]:
file_path_from = 'speaker.ann'
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0
with open("Otvety2.txt", "r", encoding="utf8") as f:
    for line in tqdm(f):
        n_ft = 0
        spls = line.split("===")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)

        counter += 1

        if counter > 50_000:
            break

ft_index.build(10)
ft_index.save('speaker.ann')
    
with open("index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

0it [00:10, ?it/s]

In [27]:
#  Загрузим индексы
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')
index_map = pd.read_pickle("index_speaker.pkl")

In [34]:
# Создадим модель продуктовых данных

shop_data = pd.read_csv("ProductsDataset.csv")
#shop_data = shop_data.iloc[:5000, :]

shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].progress_apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

  0%|          | 0/35548 [00:00<?, ?it/s]

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [35]:
# Подготовка для создания модели для определения домена данных

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [36]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
# Вопрос-ответный домен
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in tqdm(idxs)]
# Продуктовый домен
positive_texts = [" ".join(val) for val in tqdm(shop_data['text'].values)]

  0%|          | 0/25357 [00:00<?, ?it/s]

  0%|          | 0/35548 [00:00<?, ?it/s]

In [37]:
# ВО = 0, Прод = 1

dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, 
                                                    stratify=labels, random_state=13)

In [39]:
# Модель

x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

d:\virt_dev\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [40]:
# Качество

from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9778343321566374

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [43]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}

In [47]:
# Создаем Индексы для продуктовых данных

file_path_from = 'shop.ann'
if not os.path.isfile(file_path_from):  
    
    
    ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

    midf = np.mean(tfidf_vect.idf_)

    index_map_shop = {}
    counter = 0

    for i in tqdm(range(len(shop_data))):
        n_ft = 0
        index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
        vector_ft = np.zeros(100)
        for word in shop_data.loc[i, "text"]:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word] * idfs.get(word, midf)
                n_ft += idfs.get(word, midf)
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index_shop.add_item(counter, vector_ft)
        counter += 1

    ft_index_shop.build(10)
    ft_index_shop.save('shop.ann')

    file_path_from = 'index_shop.pkl'
    if not os.path.isfile(file_path_from):  
    
        with open("index_shop.pkl", "wb") as f:
            pickle.dump(index_map_shop, f)

  0%|          | 0/35548 [00:00<?, ?it/s]

In [48]:
# Загрузим индексы

midf = np.mean(tfidf_vect.idf_)

ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('shop.ann') 

index_map_shop = pd.read_pickle("index_shop.pkl")

In [49]:
# Основная функция преобразования текста в вектор х100

def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [51]:

updater = Updater("6728330934:AAGuAYeHKl9WqdWno_NC9b_eXZoM4KaCf8Q", use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Привет, давай пообщаемся?')

def textMessage(update, context):
    
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            context.bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    
    # 
    if distances[0] > 100.5:
        print(distances[0])
        context.bot.send_message(chat_id=update.message.chat_id, text="Моя твоя не понимать")
        return
    
    context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])
        
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

C:\Users\Пользователь\AppData\Local\Temp\ipykernel_15312\2571992157.py:40: TelegramDeprecationWarning: The argument `clean` of `start_polling` is deprecated. Please use `drop_pending_updates` instead.
  updater.start_polling(clean=True)
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "d:\virt_dev\.venv\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\connectionpool.py", line 402, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "d:\virt_dev\.venv\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\connectionpool.py", line 398, in _make_request
    httplib_response = conn.getresponse()
  File "d:\python_3.9\lib\http\client.py", line 1347, in getresponse
    response.begin()
  File "d:\python_3.9\lib\http\client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "d:\python_3.9\lib\http\client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLIN